In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data = pd.read_excel('Tickets_dataset.xlsx')
data.head()

,Issue key,Issue id,Summary,Assignee Id,Reporter,Reporter Id,Status,Created,Updated,Ops Product Name,Ops Platform,Comments,Resolved,Duration
0,OR-119683,832027,Report Domain: my-iris.co.uk Submitter: protec...,602d126f5f99cc0068232a55,dmarcreport@microsoft.com,qm:482cfe09-0141-40ba-9af8-49dfe7fa0867:4548db...,Closed,2025-03-01 02:18:00,2025-03-06 09:01:00,Automated Ticket,Automated Ticket,01/Mar/25 2:18 AM;qm:482cfe09-0141-40ba-9af8-4...,2025-03-03 07:57:00,2 days 05 hrs 39 mins
1,OR-119685,832030,Automatic renewal of taxfiler.co.uk succeeded,602d126f5f99cc0068232a55,noreply@registrar.amazon,qm:482cfe09-0141-40ba-9af8-49dfe7fa0867:441f06...,Closed,2025-03-01 02:40:00,2025-03-06 09:01:00,Automated Ticket,Automated Ticket,NaN,2025-03-03 07:58:00,2 days 05 hrs 18 mins
2,OR-119686,832031,"[Datadog] [P1] [Triggered on {device:d:,host:7...",712020:b22c4907-19ff-49bf-aa14-29905602ecd9,System,5dd64082af96bc0efbe55103,Closed,2025-03-01 03:03:00,2025-03-06 09:19:00,Automated Ticket,DataDog - (please select a PRODUCT),01/Mar/25 12:50 PM;5dd64082af96bc0efbe55103;[S...,2025-03-03 09:18:00,2 days 06 hrs 15 mins
3,OR-119687,832032,[P1] Triggered: MEPW: CORE-C&D-Drives low disk...,712020:d67bb499-12c3-4e75-9c52-8ca6de5a6507,alert@dtdg.eu,qm:482cfe09-0141-40ba-9af8-49dfe7fa0867:7c5c18...,Closed,2025-03-01 03:03:00,2025-03-06 09:29:00,Automated Ticket,Automated Ticket,03/Mar/25 7:55 AM;712020:d67bb499-12c3-4e75-9c...,2025-03-03 09:28:00,2 days 06 hrs 25 mins
4,OR-119688,832033,[Preview] Report Domain: my-iris.co.uk Submitt...,602d126f5f99cc0068232a55,dmarcreport@microsoft.com,qm:482cfe09-0141-40ba-9af8-49dfe7fa0867:4548db...,Closed,2025-03-01 03:37:00,2025-03-06 09:01:00,Automated Ticket,Automated Ticket,01/Mar/25 3:37 AM;qm:482cfe09-0141-40ba-9af8-4...,2025-03-03 07:57:00,2 days 04 hrs 20 mins


In [ ]:
data.columns

Index(['Issue key', 'Issue id', 'Summary', 'Assignee Id', 'Reporter',
       'Reporter Id', 'Status', 'Created', 'Updated', 'Ops Product Name',
       'Ops Platform', 'Comments', 'Resolved', 'Duration'],
      dtype='object')

In [ ]:
print(data['Ops Product Name'].unique())

['Automated Ticket' nan 'iSAMS' 'Other' 'Elements' 'ParentMail'
 'Websites (non-product)' 'FMP' 'HR API' 'Cascade HR' 'Staffology Payroll'
 'Networx' 'iSAMS \\ Ed:gen' 'Elements Platform' 'Innervision LPM'
 'IRIS Analytics' 'Payrite' 'iChannel' 'Practice Engine' 'Elements AML'
 'IRIS Firm Management' 'Innervision Enterprise' 'Elements Tax & Accounts'
 'Virtual Payroll Assistant' 'KashFlow Payroll' 'Elements Datum'
 'Every Compliance' 'Elements Accounts Production' 'DataPlan' 'Open Enrol'
 'Data Extractor' 'Staffology HR' 'Integration (License services)'
 'iSams Central' 'myEpayWindow' 'Elements Practice Management'
 'Bureau Services' 'OpenSpace' 'IRIS Employee Verification '
 'School Spider' 'Taxfiler Classic' 'IRIS Biostore Portal'
 'Innervision Lite' 'AccountantsWorld' 'Elements Client & Core'
 'SwipeClock' 'IRIS Reach' 'Apex HCM' 'Star Practice Management'
 'Senta Classic' 'IRIS Expenses' 'BioStore' 'Amity' 'Payrite Hosted'
 'PS Purchasing']


In [ ]:
data = data.drop_duplicates(subset = 'Issue id')

In [ ]:
print(data.isnull().sum())

Issue key              0
Issue id               0
Summary                0
Assignee Id            0
Reporter             150
Reporter Id          150
Status                 0
Created                0
Updated                0
Ops Product Name     598
Ops Platform         822
Comments            2178
Resolved             224
Duration             224
dtype: int64


In [ ]:
# Dropping missing values in Reporter,Reporter Id
data = data.dropna(subset=['Reporter','Reporter Id']).copy()

In [ ]:
data['Ops Product Name'] = data['Ops Product Name'].fillna('Unknown')
data['Ops Platform'] = data['Ops Platform'].fillna('Unknown')
data['Comments'] = data['Comments'].fillna('')

In [ ]:
data['Created'] = pd.to_datetime(data['Created'])
data['Updated'] = pd.to_datetime(data['Updated'])
data['Resolved'] = pd.to_datetime(data['Resolved'])

In [ ]:
# Removing the extra spaces in string columns
string_columns = ['Summary','Assignee Id','Reporter','Ops Product Name','Ops Platform','Status']
for col in string_columns:
  data[col] = data[col].str.strip()

In [ ]:
# Function to convert duration column values into numbers
def convert_duration(duration_str):
    if pd.isnull(duration_str):
        return np.nan

    days = 0
    hours = 0
    minutes = 0

    day_match = re.search(r'(\d+)\s*days?', duration_str)
    hour_match = re.search(r'(\d+)\s*hrs?', duration_str)
    min_match = re.search(r'(\d+)\s*mins?', duration_str)

    if day_match:
        days = int(day_match.group(1))
    if hour_match:
        hours = int(hour_match.group(1))
    if min_match:
        minutes = int(min_match.group(1))

    total_hours = days * 24 + hours + minutes / 60
    return total_hours


In [ ]:
data['Duration_Hours'] = data['Duration'].apply(convert_duration)
data['Duration_Days'] = data['Duration_Hours'] / 24

In [ ]:
print(data[['Duration', 'Duration_Hours', 'Duration_Days']].head())

                Duration  Duration_Hours  Duration_Days
0  2 days 05 hrs 39 mins       53.650000       2.235417
1  2 days 05 hrs 18 mins       53.300000       2.220833
2  2 days 06 hrs 15 mins       54.250000       2.260417
3  2 days 06 hrs 25 mins       54.416667       2.267361
4  2 days 04 hrs 20 mins       52.333333       2.180556


In [ ]:
print(data.isnull().sum())

Issue key             0
Issue id              0
Summary               0
Assignee Id           0
Reporter              0
Reporter Id           0
Status                0
Created               0
Updated               0
Ops Product Name      0
Ops Platform          0
Comments              0
Resolved            224
Duration            224
Duration_Hours      224
Duration_Days       224
dtype: int64


In [ ]:
print(data['Status'].unique())

['Closed' 'Done' 'Resolved' 'Open' 'Pending' 'Waiting for customer'
 'Work in progress' 'Waiting for support' 'In Progress']


In [ ]:
# Mapping the status column values to Open and Resolved for clear visualization
status_mapping = {
    'Open':'Open',
    'Pending':'Open',
    'Waiting for customer':'Open',
    'Work in progress':'Open',
    'Waiting for support':'Open',
    'In Progress':'Open',
    'Closed':'Resolved',
    'Done':'Resolved',
    'Resolved':'Resolved'
}

In [ ]:
data['Updated Status'] = data['Status'].map(status_mapping)

In [ ]:
print(data['Updated Status'].unique())

['Resolved' 'Open']


In [ ]:
print(data['Updated Status'].value_counts())

Updated Status
Resolved    6767
Open          41
Name: count, dtype: int64


In [ ]:
data.to_excel("cleaned_data.xlsx", index=False)
data.head()

,Issue key,Issue id,Summary,Assignee Id,Reporter,Reporter Id,Status,Created,Updated,Ops Product Name,Ops Platform,Comments,Resolved,Duration,Duration_Hours,Duration_Days,Updated Status
0,OR-119683,832027,Report Domain: my-iris.co.uk Submitter: protec...,602d126f5f99cc0068232a55,dmarcreport@microsoft.com,qm:482cfe09-0141-40ba-9af8-49dfe7fa0867:4548db...,Closed,2025-03-01 02:18:00,2025-03-06 09:01:00,Automated Ticket,Automated Ticket,01/Mar/25 2:18 AM;qm:482cfe09-0141-40ba-9af8-4...,2025-03-03 07:57:00,2 days 05 hrs 39 mins,53.650000,2.235417,Resolved
1,OR-119685,832030,Automatic renewal of taxfiler.co.uk succeeded,602d126f5f99cc0068232a55,noreply@registrar.amazon,qm:482cfe09-0141-40ba-9af8-49dfe7fa0867:441f06...,Closed,2025-03-01 02:40:00,2025-03-06 09:01:00,Automated Ticket,Automated Ticket,,2025-03-03 07:58:00,2 days 05 hrs 18 mins,53.300000,2.220833,Resolved
2,OR-119686,832031,"[Datadog] [P1] [Triggered on {device:d:,host:7...",712020:b22c4907-19ff-49bf-aa14-29905602ecd9,System,5dd64082af96bc0efbe55103,Closed,2025-03-01 03:03:00,2025-03-06 09:19:00,Automated Ticket,DataDog - (please select a PRODUCT),01/Mar/25 12:50 PM;5dd64082af96bc0efbe55103;[S...,2025-03-03 09:18:00,2 days 06 hrs 15 mins,54.250000,2.260417,Resolved
3,OR-119687,832032,[P1] Triggered: MEPW: CORE-C&D-Drives low disk...,712020:d67bb499-12c3-4e75-9c52-8ca6de5a6507,alert@dtdg.eu,qm:482cfe09-0141-40ba-9af8-49dfe7fa0867:7c5c18...,Closed,2025-03-01 03:03:00,2025-03-06 09:29:00,Automated Ticket,Automated Ticket,03/Mar/25 7:55 AM;712020:d67bb499-12c3-4e75-9c...,2025-03-03 09:28:00,2 days 06 hrs 25 mins,54.416667,2.267361,Resolved
4,OR-119688,832033,[Preview] Report Domain: my-iris.co.uk Submitt...,602d126f5f99cc0068232a55,dmarcreport@microsoft.com,qm:482cfe09-0141-40ba-9af8-49dfe7fa0867:4548db...,Closed,2025-03-01 03:37:00,2025-03-06 09:01:00,Automated Ticket,Automated Ticket,01/Mar/25 3:37 AM;qm:482cfe09-0141-40ba-9af8-4...,2025-03-03 07:57:00,2 days 04 hrs 20 mins,52.333333,2.180556,Resolved
